# Verify a FHIR Digitally Signing FHIR Bundle or QuestionnaireResponse Object

This is a Jupyter Notebook using Python 3.7 and openSSl to create JSON Web Signature (JWS)(see RFC 7515) and attach it to a FHIR Bundle or QuestionnaireResponse resource.

- If the resource is a Bundle use Bundle.signature
- If the resource is a QuestionnaireResponse use its [Signature extension](http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature
)

See enveloped signatures: http://build.fhir.org/signatures.html

*Although self-signed certificates are used for the purpose of these examples, they are not recommended for production systems.*

### Fetch Signed Bundle otr QR from FHIR Server

using AIDBox for now at `https://argopatientlist.aidbox.app/fhir/


In [ ]:
from requests import get
from json import dumps

bundle_id = "f50e72a6-290e-44f8-9b6f-adf19713d0d4/_history/183"  # insert bundle id here

url = "https://argopatientlist.aidbox.app/fhir/Bundle"
# url = 'http://test.fhir.org/r4/Bundle'
username = "basic"
password = "secret"
headers = {"Accept": "application/fhir+json" , "Content-Type": "application/fhir+json"}

r = get(f'{url}/{bundle_id}', auth=(username, password), headers = headers)
my_obj = r.json()
print("="*80)
print("STATUS: ",r.status_code)

print("="*80)
print("HEADERS:\n")
for k,v in r.headers.items():
    print(f'{k} = {v}')
print("="*80)
print("BODY:\n")
print(dumps(my_obj,indent=2))


### Or Alternatively Fetch JSON From a Local File

In [53]:
from pathlib import Path
from json import dumps, loads

in_path = Path(r'in_files/signed_object.json')
print(f'Fetching signed object from {in_path}')
my_obj = loads(in_path.read_text())
my_obj

Fetching signed object from in_files/signed_object.json


{'author': {'identifier': {'system': 'http://hl7.org/fhir/sid/us-npi',
   'value': '9941339100'}},
 'authored': '2022-06-17',
 'item': [{'answer': [{'valueString': 'Examplitis'}],
   'linkId': '1',
   'text': 'Relevant Patient Diagnoses (conditions that might be expected to improve with oxygen therapy)'},
  {'answer': [{'valueCoding': {'code': '4',
      'display': 'Replacement',
      'system': 'http://example.org'}}],
   'linkId': '2',
   'text': 'Order Reason'}],
 'questionnaire': 'http://example.org/cdex-questionnaire-example2',
 'resourceType': 'QuestionnaireResponse',
 'status': 'completed',
 'subject': {'display': 'Amy Shaw',
  'identifier': {'system': 'http://example.org/cdex/payer/member-ids',
   'type': {'coding': [{'code': 'UMB',
      'display': 'Member Number',
      'system': 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp'}],
    'text': 'Member Number'},
   'use': 'usual',
   'value': 'Member123'}},
 'id': 'cdex-questionnaireresponse-example4',
 'meta': {'prof

## Receiver/Verifier Steps

###  1. Remove the Signature from the Bundle or QR

In [54]:
def  pop_element(resource, element):
  try:
    my_element = resource.pop(element) # remove element
    return my_element 
  except KeyError:
    pass
 
if my_obj['resourceType'] == 'Bundle':
  recd_signature = pop_element(my_obj, 'signature') #for Bundles
elif my_obj['resourceType'] == 'QuestionnaireResponse':
  try:
    for i, extension in enumerate(my_obj['extension']):
       if extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature':
          my_obj_signature_ext = my_obj['extension'].pop(i) # remove element
          recd_signature = my_obj_signature_ext['valueSignature']
    if i == 0:
      my_obj.pop('extension') # remove extension if empty
  except KeyError:
    print('No signature extension found')
else:
  print('Not a Bundle or QuestionnaireResponse')

recd_signature

{'type': [{'system': 'urn:iso-astm:E1762-95:2013',
   'code': '1.2.840.10065.1.12.1.5',
   'display': 'Verification Signature'}],
 'when': '2022-06-17T22:42:19-07:00',
 'who': {'identifier': {'system': 'http://hl7.org/fhir/sid/us-npi',
   'value': '9941339108'}},
 'onBehalfOf': {'identifier': {'system': 'http://hl7.org/fhir/sid/us-npi',
   'value': '1184932014'}},
 'data': 'ZXlKaGJHY2lPaUpTVXpJMU5pSXNJbXQwZVNJNklsSlRJaXdpZEhsd0lqb2lTbGRVSWl3aWVEVmpJanBiSWsxSlNVVXpla05EUVRCbFowRjNTVUpCWjBsS1FVOUxSbGwyVFhkU0szbFJUVUV3UjBOVGNVZFRTV0l6UkZGRlFrTjNWVUZOU1VkT1RWRnpkME5SV1VSV1VWRkhSWGRLVmxWNlJWUk5Ra1ZIUVRGVlJVTkJkMHRSTWtaellWZGFkbU50TlhCWlZFVlRUVUpCUjBFeFZVVkNkM2RLVlRKR01XTXlSbk5oV0ZKMlRWSlZkMFYzV1VSV1VWRkxSRUY0U1ZwWFJuTmtSMmhzVWtkR01GbFVSWGhHZWtGV1FtZE9Wa0pCVFUxRWExWjVZVmROWjFOSFJtaGplWGRuVWtaYVRrMVRWWGRKZDFsS1MyOWFTV2gyWTA1QlVXdENSbWhhYkdGSFJtaGpNRUp2V2xkR2MyUkhhR3hhUjBZd1dWUkZkV0l6U201TlFqUllSRlJKZUUxVVFYbE9la1V6VGtSSmQwNUdiMWhFVkVsNVRWUkJlVTFxUlROT1JFbDNUa1p2ZDJkWk1IaERla0ZLUW1kT1ZrSkJXVlJ

### 2. Canonicalize the resource using IETF JSON Canonicalization Scheme (JCS):

- Remove the id and meta elements if present before canonicalization

In [55]:
from jcs import canonicalize #package for a JCS (RFC 8785) compliant canonicalizer.
my_obj_id = pop_element(my_obj, 'id')
my_obj_meta = pop_element(my_obj, 'meta')
canonical_obj = canonicalize(my_obj)
canonical_obj, len(canonical_obj)

(b'{"author":{"identifier":{"system":"http://hl7.org/fhir/sid/us-npi","value":"9941339100"}},"authored":"2022-06-17","item":[{"answer":[{"valueString":"Examplitis"}],"linkId":"1","text":"Relevant Patient Diagnoses (conditions that might be expected to improve with oxygen therapy)"},{"answer":[{"valueCoding":{"code":"4","display":"Replacement","system":"http://example.org"}}],"linkId":"2","text":"Order Reason"}],"questionnaire":"http://example.org/cdex-questionnaire-example2","resourceType":"QuestionnaireResponse","status":"completed","subject":{"display":"Amy Shaw","identifier":{"system":"http://example.org/cdex/payer/member-ids","type":{"coding":[{"code":"UMB","display":"Member Number","system":"http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp"}],"text":"Member Number"},"use":"usual","value":"Member123"}}}',
 823)

### 3. Transform canonicalize Bundle to a base64 format using the Base64-URL algorithm.

In [56]:
from base64 import urlsafe_b64encode
recd_b64_canonical_obj  = urlsafe_b64encode(canonical_obj).decode()
recd_b64_canonical_obj = recd_b64_canonical_obj.replace("=","")  #remove padding since decode package doesn't use them 
recd_b64_canonical_obj

'eyJhdXRob3IiOnsiaWRlbnRpZmllciI6eyJzeXN0ZW0iOiJodHRwOi8vaGw3Lm9yZy9maGlyL3NpZC91cy1ucGkiLCJ2YWx1ZSI6Ijk5NDEzMzkxMDAifX0sImF1dGhvcmVkIjoiMjAyMi0wNi0xNyIsIml0ZW0iOlt7ImFuc3dlciI6W3sidmFsdWVTdHJpbmciOiJFeGFtcGxpdGlzIn1dLCJsaW5rSWQiOiIxIiwidGV4dCI6IlJlbGV2YW50IFBhdGllbnQgRGlhZ25vc2VzIChjb25kaXRpb25zIHRoYXQgbWlnaHQgYmUgZXhwZWN0ZWQgdG8gaW1wcm92ZSB3aXRoIG94eWdlbiB0aGVyYXB5KSJ9LHsiYW5zd2VyIjpbeyJ2YWx1ZUNvZGluZyI6eyJjb2RlIjoiNCIsImRpc3BsYXkiOiJSZXBsYWNlbWVudCIsInN5c3RlbSI6Imh0dHA6Ly9leGFtcGxlLm9yZyJ9fV0sImxpbmtJZCI6IjIiLCJ0ZXh0IjoiT3JkZXIgUmVhc29uIn1dLCJxdWVzdGlvbm5haXJlIjoiaHR0cDovL2V4YW1wbGUub3JnL2NkZXgtcXVlc3Rpb25uYWlyZS1leGFtcGxlMiIsInJlc291cmNlVHlwZSI6IlF1ZXN0aW9ubmFpcmVSZXNwb25zZSIsInN0YXR1cyI6ImNvbXBsZXRlZCIsInN1YmplY3QiOnsiZGlzcGxheSI6IkFteSBTaGF3IiwiaWRlbnRpZmllciI6eyJzeXN0ZW0iOiJodHRwOi8vZXhhbXBsZS5vcmcvY2RleC9wYXllci9tZW1iZXItaWRzIiwidHlwZSI6eyJjb2RpbmciOlt7ImNvZGUiOiJVTUIiLCJkaXNwbGF5IjoiTWVtYmVyIE51bWJlciIsInN5c3RlbSI6Imh0dHA6Ly9obDcub3JnL2ZoaXIvdXMvZGF2aW5jaS1ocmV4L0NvZGVTeXN0ZW0

### 4. Get the base64 encoded JWS  from the `Bundle.signature.data`  element

In [57]:
recd_b64_jws = recd_signature['data']
recd_b64_jws

'ZXlKaGJHY2lPaUpTVXpJMU5pSXNJbXQwZVNJNklsSlRJaXdpZEhsd0lqb2lTbGRVSWl3aWVEVmpJanBiSWsxSlNVVXpla05EUVRCbFowRjNTVUpCWjBsS1FVOUxSbGwyVFhkU0szbFJUVUV3UjBOVGNVZFRTV0l6UkZGRlFrTjNWVUZOU1VkT1RWRnpkME5SV1VSV1VWRkhSWGRLVmxWNlJWUk5Ra1ZIUVRGVlJVTkJkMHRSTWtaellWZGFkbU50TlhCWlZFVlRUVUpCUjBFeFZVVkNkM2RLVlRKR01XTXlSbk5oV0ZKMlRWSlZkMFYzV1VSV1VWRkxSRUY0U1ZwWFJuTmtSMmhzVWtkR01GbFVSWGhHZWtGV1FtZE9Wa0pCVFUxRWExWjVZVmROWjFOSFJtaGplWGRuVWtaYVRrMVRWWGRKZDFsS1MyOWFTV2gyWTA1QlVXdENSbWhhYkdGSFJtaGpNRUp2V2xkR2MyUkhhR3hhUjBZd1dWUkZkV0l6U201TlFqUllSRlJKZUUxVVFYbE9la1V6VGtSSmQwNUdiMWhFVkVsNVRWUkJlVTFxUlROT1JFbDNUa1p2ZDJkWk1IaERla0ZLUW1kT1ZrSkJXVlJCYkZaVVRWSk5kMFZSV1VSV1VWRkpSRUZ3UkZsWGVIQmFiVGw1WW0xc2FFMVNTWGRGUVZsRVZsRlJTRVJCYkZSWldGWjZXVmQ0Y0dSSE9IaEdWRUZVUW1kT1ZrSkJiMDFFUldoc1dWZDRNR0ZIVmtWWldGSm9UVlJGV0UxQ1ZVZEJNVlZGUVhkM1QxSllTbkJaZVVKSldWZEdla3hEUWtWV2F6QjRTbFJCYWtKbmEzRm9hMmxIT1hjd1FrTlJSVmRHYlZadldWZEdlbEZIYUd4WlYzZ3dZVWRXYTFsWVVtaE5VelYyWTIxamQyZG5SMmxOUVRCSFExTnhSMU5KWWpORVVVVkNRVkZWUVVFMFNVSnFkMEYzWjJkSFM

### 5. Base64 decode the encoded JWS

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing

In [58]:
labels = ['header', 'payload', 'signature']
from base64 import b64decode
recd_jws = b64decode(recd_b64_jws.encode()).decode()
for i,j in enumerate(recd_jws.split('.')):
    print(f'{labels[i]}:')
    print(f'{j}')
    print()

header:
eyJhbGciOiJSUzI1NiIsImt0eSI6IlJTIiwidHlwIjoiSldUIiwieDVjIjpbIk1JSUUzekNDQTBlZ0F3SUJBZ0lKQU9LRll2TXdSK3lRTUEwR0NTcUdTSWIzRFFFQkN3VUFNSUdOTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVJVd0V3WURWUVFLREF4SVpXRnNkR2hsUkdGMFlURXhGekFWQmdOVkJBTU1Ea1Z5YVdNZ1NHRmhjeXdnUkZaTk1TVXdJd1lKS29aSWh2Y05BUWtCRmhabGFHRmhjMEJvWldGc2RHaGxaR0YwWVRFdWIzSm5NQjRYRFRJeE1UQXlOekUzTkRJd05Gb1hEVEl5TVRBeU1qRTNOREl3TkZvd2dZMHhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhGVEFUQmdOVkJBb01ERWhsWVd4MGFHVkVZWFJoTVRFWE1CVUdBMVVFQXd3T1JYSnBZeUJJWVdGekxDQkVWazB4SlRBakJna3Foa2lHOXcwQkNRRVdGbVZvWVdGelFHaGxZV3gwYUdWa1lYUmhNUzV2Y21jd2dnR2lNQTBHQ1NxR1NJYjNEUUVCQVFVQUE0SUJqd0F3Z2dHS0FvSUJnUURwS2NTa29BTTZzVzIxK3ZXVGVJVk9HeDEwTVdhc1F5N1ZIaWQ2enlxWEFCTSt6bmZCblhlbnlVMGoxRlR2UG1SZk9Eb09EWFZ1UFV3RG9taENIaCtiY2xXOUtNMm81NjNjeFJLRXZCbmFIcnNqdzV5Tm14TzVZakVSYmh0SGRRZXFrdGR3M1ZZRVJSOUhveExPM0Zrc3pSMjkySFRCNHhXM3lXbFYzZ1RrTVFvelBT

### 6. reattach the payload - the base64 encoded Bundle or QR - into the JWS payload element. 

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing

In [59]:
split_sig = recd_jws.split('.')
split_sig[1] = recd_b64_canonical_obj
recd_jws = '.'.join(split_sig)
for i,j in enumerate(recd_jws.split('.')):
    print(f'{labels[i]}:')
    print(f'{j}')
    print()

header:
eyJhbGciOiJSUzI1NiIsImt0eSI6IlJTIiwidHlwIjoiSldUIiwieDVjIjpbIk1JSUUzekNDQTBlZ0F3SUJBZ0lKQU9LRll2TXdSK3lRTUEwR0NTcUdTSWIzRFFFQkN3VUFNSUdOTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVJVd0V3WURWUVFLREF4SVpXRnNkR2hsUkdGMFlURXhGekFWQmdOVkJBTU1Ea1Z5YVdNZ1NHRmhjeXdnUkZaTk1TVXdJd1lKS29aSWh2Y05BUWtCRmhabGFHRmhjMEJvWldGc2RHaGxaR0YwWVRFdWIzSm5NQjRYRFRJeE1UQXlOekUzTkRJd05Gb1hEVEl5TVRBeU1qRTNOREl3TkZvd2dZMHhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhGVEFUQmdOVkJBb01ERWhsWVd4MGFHVkVZWFJoTVRFWE1CVUdBMVVFQXd3T1JYSnBZeUJJWVdGekxDQkVWazB4SlRBakJna3Foa2lHOXcwQkNRRVdGbVZvWVdGelFHaGxZV3gwYUdWa1lYUmhNUzV2Y21jd2dnR2lNQTBHQ1NxR1NJYjNEUUVCQVFVQUE0SUJqd0F3Z2dHS0FvSUJnUURwS2NTa29BTTZzVzIxK3ZXVGVJVk9HeDEwTVdhc1F5N1ZIaWQ2enlxWEFCTSt6bmZCblhlbnlVMGoxRlR2UG1SZk9Eb09EWFZ1UFV3RG9taENIaCtiY2xXOUtNMm81NjNjeFJLRXZCbmFIcnNqdzV5Tm14TzVZakVSYmh0SGRRZXFrdGR3M1ZZRVJSOUhveExPM0Zrc3pSMjkySFRCNHhXM3lXbFYzZ1RrTVFvelBT

### 7. Obtain public Key from the first certificate in JWS header `"x5c"` key

- base64 decode the key value
- Verify Issuer, Validity Dates, Subject,and KeyUsage of certificate,
- Get the “Subject Public Key Info” from the cert to verify the signature

In [60]:
from jose import jws
recd_header = jws.get_unverified_header(recd_jws)
recd_header

{'alg': 'RS256',
 'kty': 'RS',
 'typ': 'JWT',
 'x5c': ['MIIE3zCCA0egAwIBAgIJAOKFYvMwR+yQMA0GCSqGSIb3DQEBCwUAMIGNMQswCQYDVQQGEwJVUzETMBEGA1UECAwKQ2FsaWZvcm5pYTESMBAGA1UEBwwJU2F1c2FsaXRvMRUwEwYDVQQKDAxIZWFsdGhlRGF0YTExFzAVBgNVBAMMDkVyaWMgSGFhcywgRFZNMSUwIwYJKoZIhvcNAQkBFhZlaGFhc0BoZWFsdGhlZGF0YTEub3JnMB4XDTIxMTAyNzE3NDIwNFoXDTIyMTAyMjE3NDIwNFowgY0xCzAJBgNVBAYTAlVTMRMwEQYDVQQIDApDYWxpZm9ybmlhMRIwEAYDVQQHDAlTYXVzYWxpdG8xFTATBgNVBAoMDEhlYWx0aGVEYXRhMTEXMBUGA1UEAwwORXJpYyBIYWFzLCBEVk0xJTAjBgkqhkiG9w0BCQEWFmVoYWFzQGhlYWx0aGVkYXRhMS5vcmcwggGiMA0GCSqGSIb3DQEBAQUAA4IBjwAwggGKAoIBgQDpKcSkoAM6sW21+vWTeIVOGx10MWasQy7VHid6zyqXABM+znfBnXenyU0j1FTvPmRfODoODXVuPUwDomhCHh+bclW9KM2o563cxRKEvBnaHrsjw5yNmxO5YjERbhtHdQeqktdw3VYERR9HoxLO3FkszR292HTB4xW3yWlV3gTkMQozPScJKH3bG8Pqq6AYPJ7C4YBIlUSdBMVl3qneEfg7fuxiFfXofdTVm7rMiiG7X9z43PfilqaeisfmtRxAlRwENXrH3OvODPyL0rTnG8CsbAXYVIMmddHe4ZF9plh95sj4pMThLtcJX/o9XHLjh7EmZygJHWEQq4PwFwZdmbcfhCmOr88H8BbUru/7V6zbsG1N1CWlndlbVznL+3IMOr8kXaHcanqfckgFV4ErnfjFJq1OIaAmsMj85xMk

In [61]:
recd_cert = b64decode(recd_header['x5c'][0])
with open('recd_cert.der', 'wb') as f:
    f.write(recd_cert)
!openssl x509 -in recd_cert.der -inform DER -text

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number: 16322561221100825744 (0xe28562f33047ec90)
    Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=US, ST=California, L=Sausalito, O=HealtheData1, CN=Eric Haas, DVM/emailAddress=ehaas@healthedata1.org
        Validity
            Not Before: Oct 27 17:42:04 2021 GMT
            Not After : Oct 22 17:42:04 2022 GMT
        Subject: C=US, ST=California, L=Sausalito, O=HealtheData1, CN=Eric Haas, DVM/emailAddress=ehaas@healthedata1.org
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (3072 bit)
                Modulus:
                    00:e9:29:c4:a4:a0:03:3a:b1:6d:b5:fa:f5:93:78:
                    85:4e:1b:1d:74:31:66:ac:43:2e:d5:1e:27:7a:cf:
                    2a:97:00:13:3e:ce:77:c1:9d:77:a7:c9:4d:23:d4:
                    54:ef:3e:64:5f:38:3a:0e:0d:75:6e:3d:4c:03:a2:
                    68:42:1e:1f:9b:72:55:bd:28:cd:a8:e7:ad:dc:c5:
       

### 10. Verify Signature using the public key or the X.509 Certificate

Alternatively:
1. visit https://jwt.io.
1. At the top of the page, select "RS256" for the algorithm.
1. Paste the JWS value printed below into the “Encoded” field.
1. The plaintext JWT will be displayed in the “Decoded:Payload” field.
1. The X509 Cert above will appear in the "Verify Signature" box.
1. If verified, a “Signature Verified” message will appear  in the bottom left hand corner.

In [62]:
recd_jws

'eyJhbGciOiJSUzI1NiIsImt0eSI6IlJTIiwidHlwIjoiSldUIiwieDVjIjpbIk1JSUUzekNDQTBlZ0F3SUJBZ0lKQU9LRll2TXdSK3lRTUEwR0NTcUdTSWIzRFFFQkN3VUFNSUdOTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVJVd0V3WURWUVFLREF4SVpXRnNkR2hsUkdGMFlURXhGekFWQmdOVkJBTU1Ea1Z5YVdNZ1NHRmhjeXdnUkZaTk1TVXdJd1lKS29aSWh2Y05BUWtCRmhabGFHRmhjMEJvWldGc2RHaGxaR0YwWVRFdWIzSm5NQjRYRFRJeE1UQXlOekUzTkRJd05Gb1hEVEl5TVRBeU1qRTNOREl3TkZvd2dZMHhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhGVEFUQmdOVkJBb01ERWhsWVd4MGFHVkVZWFJoTVRFWE1CVUdBMVVFQXd3T1JYSnBZeUJJWVdGekxDQkVWazB4SlRBakJna3Foa2lHOXcwQkNRRVdGbVZvWVdGelFHaGxZV3gwYUdWa1lYUmhNUzV2Y21jd2dnR2lNQTBHQ1NxR1NJYjNEUUVCQVFVQUE0SUJqd0F3Z2dHS0FvSUJnUURwS2NTa29BTTZzVzIxK3ZXVGVJVk9HeDEwTVdhc1F5N1ZIaWQ2enlxWEFCTSt6bmZCblhlbnlVMGoxRlR2UG1SZk9Eb09EWFZ1UFV3RG9taENIaCtiY2xXOUtNMm81NjNjeFJLRXZCbmFIcnNqdzV5Tm14TzVZakVSYmh0SGRRZXFrdGR3M1ZZRVJSOUhveExPM0Zrc3pSMjkySFRCNHhXM3lXbFYzZ1RrTVFvelBTY0pLSDN

In [63]:
!openssl x509 -in recd_cert.der -inform DER -pubkey -noout > recd-public-key.pem
with open('recd-public-key.pem') as f:
    pem_public = f.read()
pem_public

'-----BEGIN PUBLIC KEY-----\nMIIBojANBgkqhkiG9w0BAQEFAAOCAY8AMIIBigKCAYEA6SnEpKADOrFttfr1k3iF\nThsddDFmrEMu1R4nes8qlwATPs53wZ13p8lNI9RU7z5kXzg6Dg11bj1MA6JoQh4f\nm3JVvSjNqOet3MUShLwZ2h67I8OcjZsTuWIxEW4bR3UHqpLXcN1WBEUfR6MSztxZ\nLM0dvdh0weMVt8lpVd4E5DEKMz0nCSh92xvD6qugGDyewuGASJVEnQTFZd6p3hH4\nO37sYhX16H3U1Zu6zIohu1/c+Nz34pamnorH5rUcQJUcBDV6x9zrzgz8i9K05xvA\nrGwF2FSDJnXR3uGRfaZYfebI+KTE4S7XCV/6PVxy44exJmcoCR1hEKuD8BcGXZm3\nH4Qpjq/PB/AW1K7v+1es27BtTdQlpZ3ZW1c5y/tyDDq/JF2h3Gp6n3JIBVeBK534\nxSatTiGgJrDI/OcTJI8ly8nCy/7uZ9qOgYPd/1EX6rjqEiBjgkduQ2mc6cNpN7O6\nBPXTMBFl7X14GLYdm5Y8ubRR9Bo6TpMdO4+2U+R58Z5bAgMBAAE=\n-----END PUBLIC KEY-----\n'

In [64]:
try:
    verify = jws.verify(recd_jws, pem_public , algorithms=['RS256'])
    print('#     #                                                ### ')
    print('#     #  ######  #####   #  ######  #  ######  #####   ### ')
    print('#     #  #       #    #  #  #       #  #       #    #  ### ')
    print('#     #  #####   #    #  #  #####   #  #####   #    #   #  ')
    print(' #   #   #       #####   #  #       #  #       #    #      ')
    print('  # #    #       #   #   #  #       #  #       #    #  ### ')
    print('   #     ######  #    #  #  #       #  ######  #####   ### ')
    print('                                                           ')

except Exception as e:
    print('#    #   ####   #####      #    #  ######  #####   #  ######  #  ######  #####       ###          #   ')
    print('##   #  #    #    #        #    #  #       #    #  #  #       #  #       #    #       #          #    ')
    print('# #  #  #    #    #        #    #  #####   #    #  #  #####   #  #####   #    #           #####  #    ')
    print('#  # #  #    #    #        #    #  #       #####   #  #       #  #       #    #       #          #    ')
    print('#   ##  #    #    #         #  #   #       #   #   #  #       #  #       #    #      ###          #   ')
    print('#    #   ####     #          ##    ######  #    #  #  #       #  ######  #####        #            ## ')
    print('                ')
    print(f"not verified: {e}")

#     #                                                ### 
#     #  ######  #####   #  ######  #  ######  #####   ### 
#     #  #       #    #  #  #       #  #       #    #  ### 
#     #  #####   #    #  #  #####   #  #####   #    #   #  
 #   #   #       #####   #  #       #  #       #    #      
  # #    #       #   #   #  #       #  #       #    #  ### 
   #     ######  #    #  #  #       #  ######  #####   ### 
                                                           
